# Cutting the crap and getting recipes from blogs

In [16]:
from bs4 import BeautifulSoup
import requests
import json
import webbrowser
import os

In [23]:
url = 'https://therecipecritic.com/korean-ground-beef-rice-bowls/'

In [3]:
def _get_html_parser(url):
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [4]:
soup = _get_html_parser(url)

In [5]:
def _get_recipe_tag(parser):
    scripts = parser.find_all('script')
    for sc in scripts:
        if 'yoast-schema-graph' in sc.get_attribute_list('class'):
            return sc
    raise RuntimeError('Couldn\'t find recipe tag in html.')
    

In [6]:
tag = _get_recipe_tag(soup)

In [7]:
def _get_recipe_dict(tag):
    _json = json.loads(tag.string)['@graph']
    _recipe = list(filter(lambda x: x['@type'] == 'Recipe', _json))[0]
    if not _recipe:
        raise RuntimeError('Couldn\'t find recipe dictionary in json.')
    return _recipe

In [44]:
recipe = _get_recipe_dict(tag)

In [45]:
def _parse_recipe(_recipe):
    recipe = {
        new: _recipe[old] for old, new in zip(
            ['name', 'image', 'recipeIngredient', 'recipeInstructions'],
            ['title', 'images', 'ingredients', 'instructions']
        ) if old in _recipe.keys()
    }
    recipe['instructions'] = [x['text'] for x in recipe['instructions']]
    return recipe
    

In [46]:
recipe = _parse_recipe(recipe)

In [91]:
def _write_html(recipe, file = None, folder = None):
    if file is None:
        file = '_'.join(recipe['title'].split( ))
    if not file.endswith('.html'):
        file += '.html'
    if folder is None:
        folder = os.getcwd()
    filename = os.path.join(folder,file)
    scaffolding = """<html>
    <head>
    <title>{}</title>
    </head>
    <body>
    <h1>{}</h1>
    {}
    </body>
    </html> 
    """.format(recipe['title'], recipe['title'],'{}')
    ingredients = (
        "<h2>Ingredients</h2>" + 
        "\n".join(["<p>{}</p>".format(x) for x in recipe['ingredients']])
    ) if 'ingredients' in recipe.keys() else ""
    instructions =(
        "<h2>Instructions</h2>" + 
        "\n".join(["<p>{}</p>".format(x) for x in recipe['instructions']])
    ) if 'istructions' in recipe.keys() else ""
    image = "<img src=\"{}\">".format(recipe['images'][0]) if 'images' in recipe.keys() else ""
    with open(os.path.join(filename), 'w') as f:
        f.write(
            scaffolding.format(
                ingredients +
                instructions +
                image
            )
        )
    return filename

In [92]:
html = _write_html(recipe)

In [93]:
def _open_html(html):
    webbrowser.open(html)

In [94]:
_open_html(html)

In [21]:
def givemethedamnrecipe(url, folder = None, file = None):
    parser = _get_html_parser(url)
    tag = _get_recipe_tag(parser)
    recipe = _get_recipe_dict(tag)
    html = _write_html(_parse_recipe(recipe))
    _open_html(html)

In [6]:
url = 'https://therecipecritic.com/korean-ground-beef-rice-bowls/'
url2 = 'https://therecipecritic.com/ground-beef-enchiladas/'
url3 = 'https://therecipecritic.com/slow-cooker-beef-bourguignon/'

In [22]:
givemethedamnrecipe(url)
givemethedamnrecipe(url2)
givemethedamnrecipe(url3)

HTML file created at /home/lukas/Documents/code/python/recipy/Korean_Ground_Beef_and_Rice_Bowls.html
HTML file created at /home/lukas/Documents/code/python/recipy/Salsa_Verde_Beef_and_Mushroom_Enchiladas.html
HTML file created at /home/lukas/Documents/code/python/recipy/Slow_Cooker_Beef_Bourguignon.html
